In [1]:
import pandas as pd
cleaned_df = pd.read_csv(r'C:\All_Project\finaltrain\notebooks\data\cleaned_data.csv')
cleaned_df

,Delivery_person_Age,Delivery_person_Ratings,distance,order_preparation_time,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,36.0,4.2,10.0,15.0,Fog,Jam,2,motorcycle,3.0,No,Metropolitian,46
1,21.0,4.7,6.0,10.0,Stormy,High,1,motorcycle,1.0,No,Metropolitian,23
2,23.0,4.7,13.0,10.0,Sandstorms,Medium,1,scooter,1.0,No,Metropolitian,21
3,34.0,4.3,2.0,10.0,Sandstorms,Low,0,motorcycle,0.0,No,Metropolitian,20
4,24.0,4.7,19.0,15.0,Fog,Jam,1,scooter,1.0,No,Metropolitian,41
...,...,...,...,...,...,...,...,...,...,...,...,...
45579,30.0,4.8,1.0,10.0,Windy,High,1,motorcycle,0.0,No,Metropolitian,32
45580,21.0,4.6,10.0,15.0,Windy,Jam,0,motorcycle,1.0,No,Metropolitian,36
45581,30.0,4.9,4.0,15.0,Cloudy,Low,1,scooter,0.0,No,Metropolitian,16
45582,20.0,4.7,6.0,5.0,Cloudy,High,0,motorcycle,1.0,No,Metropolitian,26


In [2]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Delivery_person_Age      45584 non-null  float64
 1   Delivery_person_Ratings  45584 non-null  float64
 2   distance                 45584 non-null  float64
 3   order_preparation_time   45584 non-null  float64
 4   Weather_conditions       45584 non-null  object 
 5   Road_traffic_density     45584 non-null  object 
 6   Vehicle_condition        45584 non-null  int64  
 7   Type_of_vehicle          45584 non-null  object 
 8   multiple_deliveries      45584 non-null  float64
 9   Festival                 45584 non-null  object 
 10  City                     45584 non-null  object 
 11  Time_taken (min)         45584 non-null  int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 4.2+ MB


In [62]:
cleaned_df.isnull().sum()

Delivery_person_Age        0
Delivery_person_Ratings    0
distance                   0
order_preparation_time     0
Weather_conditions         0
Road_traffic_density       0
Vehicle_condition          0
Type_of_vehicle            0
multiple_deliveries        0
Festival                   0
City                       0
Time_taken (min)           0
dtype: int64

In [30]:
X = cleaned_df.drop(labels=['Time_taken (min)'],axis = 1)
y = cleaned_df[['Time_taken (min)']]

In [31]:
X.shape

(45584, 11)

In [63]:
numerical_col = X.select_dtypes(exclude = 'object').columns
categorical_col = X.select_dtypes(include = 'object').columns

In [65]:
print(categorical_col)

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_vehicle',
       'Festival', 'City'],
      dtype='object')


In [66]:
print(numerical_col)

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'distance',
       'order_preparation_time', 'Vehicle_condition', 'multiple_deliveries'],
      dtype='object')


In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet # Model Training
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error # Model Evaluation metrics

In [83]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [69]:
df['Type_of_vehicle'].unique()

array(['motorcycle', 'scooter', 'electric_scooter', 'bicycle'],
      dtype=object)

In [70]:
weather_categories = ['Sunny','Stormy','Sandstorms','Windy','Cloudy','Fog']
traffic_categories = ['Low','Medium','High','Jam']
vehicleType_categories = ['motorcycle', 'scooter', 'electric_scooter', 'bicycle']
festival_categories=['No','Yes']
city_categories= ['Urban','Metropolitian','Semi-Urban']

In [75]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[weather_categories,traffic_categories,vehicleType_categories,festival_categories,city_categories])),
    ('scaler',StandardScaler())
    ]

)
preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_col),
('cat_pipeline',cat_pipeline,categorical_col)
])

In [76]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=36)

In [77]:
X_train_transformed = preprocessor.fit_transform(X_train)

# Get the feature names from the preprocessor
feature_names = preprocessor.get_feature_names_out()

# Create a DataFrame with the transformed data
X_train = pd.DataFrame(X_train_transformed, columns=feature_names)

In [78]:
X_test_transformed = preprocessor.fit_transform(X_test)

# Get the feature names from the preprocessor
feature_names = preprocessor.get_feature_names_out()

# Create a DataFrame with the transformed data
X_test = pd.DataFrame(X_test_transformed, columns=feature_names)

In [79]:
X_train

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__distance,num_pipeline__order_preparation_time,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City
0,0.000500,0.001833,1.921256,-0.368715,-1.219707,-1.310216,-0.322010,-1.093337,-0.773151,-0.14071,0.515355
1,1.660776,-0.101133,-0.589043,-0.419541,-1.219707,-1.310216,-0.903766,-0.293009,-0.773151,-0.14071,-1.864467
2,1.308765,0.508604,-0.051122,-0.394128,-0.025767,0.453323,1.423257,1.307646,-0.773151,-0.14071,0.515355
3,0.428737,-0.101133,1.921256,-0.419541,-0.025767,0.453323,0.841502,1.307646,0.759909,-0.14071,0.515355
4,-0.275285,0.508604,1.383335,-0.406834,1.168174,0.453323,0.841502,-1.093337,0.759909,-0.14071,0.515355
...,...,...,...,...,...,...,...,...,...,...,...
36462,-0.099280,-0.101133,1.921256,-0.394128,1.168174,0.453323,-0.322010,1.307646,-0.773151,-0.14071,0.515355
36463,-0.803302,-0.101133,0.128185,-0.419541,-0.025767,0.453323,1.423257,1.307646,0.759909,-0.14071,0.515355
36464,0.780748,0.203736,0.666107,-0.419541,1.168174,-1.310216,-0.322010,1.307646,0.759909,-0.14071,0.515355
36465,1.660776,0.813473,-1.126964,1.727866,-0.025767,0.002976,1.423257,0.507318,0.759909,-0.14071,0.515355


In [80]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [84]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'DecisionTreeRegressor':DecisionTreeRegressor(),
    'RandomForestRegressor' : RandomForestRegressor(),
    'SupportVectorRegressor' : SVR(kernel='linear')
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 6.163883423457684
MAE: 4.9373150710928595
R2 score 56.71173198372984


Lasso
Model Training Performance
RMSE: 6.655927819802023
MAE: 5.359363664780386
R2 score 49.52473772333167


Ridge
Model Training Performance
RMSE: 6.163882526140499
MAE: 4.937316091380807
R2 score 56.71174458724659


Elasticnet
Model Training Performance
RMSE: 6.733382589679216
MAE: 5.4365819607987715
R2 score 48.34314514322917


DecisionTreeRegressor
Model Training Performance
RMSE: 5.426715689263266
MAE: 4.125973456180761
R2 score 66.44667872763645




<ipython-input-84-63a1009985a4>:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,y_train)


RandomForestRegressor
Model Training Performance
RMSE: 4.045942701057816
MAE: 3.203061704455831
R2 score 81.34905536012191




/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SupportVectorRegressor
Model Training Performance
RMSE: 6.172147104213263
MAE: 4.926089115654014
R2 score 56.59558435609118


